# Finding Conan
We want to use Chatgpt4 to retrive information out of our scraped data - Polieimeldungen https://www.berlin.de/polizei/polizeimeldungen/

We want the following information:
1. When did the crime happened
2. What crime happened
3. Where did it happened


In [2]:
# Install in terminal
# pip install openai
# pip install --upgrade openai
# npm install openai

# **Import the necessary packages**

In [203]:
import os
import openai
import pandas as pd
from IPython.display import display, Markdown
import requests
import json
import shutil 
import time
import glob
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib as plt
from datetime import datetime

# **Set API_Key secret**

In [134]:
# Get the path where the credential is
path_cred = os.path.join(os.path.dirname(os.path.dirname(os.getcwd())), 'credentials')

# open the credential
with open(os.path.join(path_cred,'secrets.json')) as secrets_file:
    secrets = json.load(secrets_file)
    api_key = secrets['ys_api_key']

# **select a model**

https://platform.openai.com/docs/api-reference/models


# **connect to API**

# load single json file

In [136]:
path_to_json_files = '/Users/ellenlee/code/hclpush/finding-conan/raw-data/unstructured-data' # TODO
specification_1 = 'categorized-test-latest-2000-files'
specification_2 = 'one-case'
file_name = 'pressemitteilung.1207392.json'
f = open(os.path.join(path_to_json_files, specification_1, specification_2, file_name))
case = json.load(f)

del case['subtitle']

In [137]:
case

{'title': 'Vierjährige von Auto angefahren',
 'place': 'Charlottenburg-Wilmersdorf',
 'date': '17.05.2022',
 'data': [{'number': '1047',
   'description': 'Gestern Nachmittag wurde eine Vierjährige bei einem Verkehrsunfall im Ortsteil Grunewald verletzt. Nach derzeitigen Erkenntnissen und Zeugenaussagen wollte das Kind gegen 16.50 Uhr die Gehwegseite der Auerbachstraße wechseln, löste sich dafür von der Hand seiner Begleiterin, rannte auf die Straße und wurde dort von einem Auto angefahren. In Folge des Zusammenstoßes erlitt das Mädchen Verletzungen am Knie und im Bereich der Rippen, mit denen es vorsorglich in einem Krankenhaus verblieb. Da der 67-jährige Fahrzeugführer während der Sachverhaltsaufnahme geistig abwesend wirkte und nur schwer den Anweisungen der Einsatzkräfte folgen konnte, wurde ihm nach rechtlicher Belehrung ein Atemalkoholtest angeboten. Diesem stimmte er zu. Das Ergebnis: Rund ein Promille. Somit wurde sein Führerschein beschlagnahmt, sein Wagen geparkt und verschlo

In [139]:
path_to_json_files

'/Users/ellenlee/code/hclpush/finding-conan/raw-data/unstructured-data'

# Create Important Functions

In [140]:
def chatGPT(text, api_key):
    url = "https://api.openai.com/v1/completions"
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {api_key}",
    }
    prompt = { 
        "model":"text-davinci-003",
        "prompt": # TODO: add property crimes
        """
        You are crime analyzing assistant. The assistant is helpful, clever, and gives accurate answers.

        You define types of crimes according to the following categories:
        Homicide = The act of unlawfully causing the death of another person.
        Hate Crime - Disability = Anti-Mental Disability, Anti-Physical Disability
        Hate Crime - Gender = Anti-Male, Anti-Female
        Hate Crime - Gender Identity = Anti-Transgender, Anti-Gender Non-Conforming
        Hate Crime - Religious = Anti-Buddhist, Anti-Catholic, Anti-Eastern Orthodox (Russian, Greek, Other), Anti-Hindu, Anti-Islamic, Anti-Jehovah’s Witness, Anti-Jewish, Anti-Mormon, Anti-Multiple Religions, Group, Anti-Other Christian, Anti-Other Religion, Anti-Protestant, Anti-Sikh, Anti-Atheism/Agnosticism, etc.
        Hate Crime - Sexual orientation = Anti-Bisexual, Anti-Gay (Male), Anti-Heterosexual, Anti-Lesbian, Anti-Lesbian, Gay, Bisexual, or Transgender (Mixed Group)
        Hate Crime - Racial/Ethnicity = Anti-American Indian or Alaska Native, Anti-Arab, Anti-Asian, Anti-Black or African American, Anti-Hispanic or Latino, Anti-Multiple Races, Group, Anti-Native Hawaiian or Other Pacific Islander, Anti-Other Race/Ethnicity/Ancestry, Anti-White
        Property Crimes = Offenses that involve unlawful interference with someone else's property, such as theft, burglary, or vandalism.
        Verbal Abuse = The use of offensive, derogatory, or threatening language with the intention to harm or intimidate another person verbally.
        Property Damage = Is damage or destruction of real or tangible personal property, 
        Drug Offenses = Violations of laws related to the possession, distribution, manufacturing, or use of illegal substances or controlled substances without proper authorization.
        General Assault = Physical or verbal attacks on another person that result in causing harm or fear.
        Sexual Assault = Non-consensual sexual acts or behavior that involve physical or psychological harm or coercion
        Sexual Harassment = Unwanted sexual advances, comments, or conduct that creates a hostile or uncomfortable environment for the victim
        Unclassified = Cases or incidents that do not fit into any specific predefined category.

        Read the following text of a crime that happened in Berlin, Germany. Give us information about the crime in the form of a key value pair. List the type of crime described in the text according to your definition. If multiple crimes occurred add all types of crimes that are described in the text. List the location as accurate as possible.
        Also list the year, date, and time of the crime. List the sex of the victims and the sex of the offender. List how many victims and how many offenders were involved.  If have no clear answer you response with "unknown".

        Here is one example for the following crime and the desired output:

        Junge Männer mit Messer verletzt
        Polizeimeldung vom 13.05.2023
        Mitte
        Nr. 0749
        Gestern Abend zeigten ein Jugendlicher und zwei junge Männer in der Alex-Wache in Mitte Körperverletzungen, begangen aus einer Gruppe, an. Die beiden 20-Jährigen und ihr 17 Jahre alter Begleiter gaben an, dass sie gegen 21.30 Uhr auf dem Alexanderplatz in Richtung S-Bahnhof unterwegs waren, als sie plötzlich aus einer Gruppe von sechs bis acht Männern heraus zunächst angepöbelt und dann körperlich attackiert wurden. Dabei erlitt einer der 20-Jährigen eine stark blutende Schnittverletzung im Gesicht und sein gleichaltriger Bekannter eine oberflächliche Stichverletzung am Rücken. Der Jugendliche blieb unverletzt. Alarmierte Rettungskräfte brachten die beiden Verletzten in Krankenhäuser. Der am Rücken Verletzte wurde ambulant behandelt, der im Gesicht Verletzte stationär aufgenommen. Einsatzkräfte stellten kurz nach der Anzeigenerstattung in Höhe der Sankt Marienkirche zwei 19-jährige Männer fest, die nach Angaben des Trios zu der angreifenden Gruppe gehörten. Die Kräfte nahmen die beiden Tatverdächtigen fest und führten sie einer erkennungsdienstlichen Behandlung zu. Nach den polizeilichen Maßnahmen wurden beide entlassen. Die Ermittlungen zum Tatgeschehen und zu den anderen Mitgliedern der Gruppe dauern an und wurden von einem Fachkommissariat der Polizeidirektion 5 (City) übernommen.
        Year: 2023
        Date: 13.05
        Time:  21.30 Uhr 
        Type of Crime: General Assault, Verbal Abuse
        Location: Alexanderplatz 
        Victim's Sex: Male
        Offender's Sex: Male
        Number of Victims: 3
        Number of Offenders: 6-8

        Please do this for the following crime:
        """ + str(text),
        "temperature":0.22,
        "max_tokens":1788,
        "top_p":0.46,
        "frequency_penalty":0,
        "presence_penalty":0
    }
    response = requests.post(url, headers=headers, json=prompt)
#     print('response: ')
#     print(response)
    print(f'======== Response.json(): {str(response.json())} ========')
    output = response.json()['choices'][0]['text'] 
    
    return output

In [173]:
def label_cases_with_gpt(num_case, files_gen, output_dict):
    """
    Calls openai api to label the raw scraped JSON files on criminal cases
    Outputs a dictionary with keys being an unique identifier (combination of index and original case number) 
        and values being the listed case info
    """
    for index, file_path in enumerate(files_gen[:num_case]): # DEBUG
        data = open(file_path)# Get labeling from GPT API
        case = json.load(data)
        del case['subtitle'] # This does not include relevant info
        
        file_name = file_path.split('/')[-1]
        case_id = str(file_name[-12:-5])
        print(f'======== Working on lebaling case number {count+1} with GPT: {case_id} ========')
        
        try: 
            raw_gpt_output = chatGPT(case, api_key)
            output_dict[case_id] = raw_gpt_output

        # Move file to folder
#         destination_dir_path = os.path.join(path_to_json_files, 'info-extracted-cases')
#         if not os.path.exists(destination_dir_path):
#             os.makedirs(destination_dir_path)
#         shutil.move(file_path, os.path.join(destination_dir_path, file_name))
        except:
            print(f'!!!!!======== Errors occurs while labeling case number {count+1} with case id {case_id} ========!!!!!')
            break
        
    return output_dict

In [159]:
def inject_cases_info_to_dict(input_dict):
    """
    Creates a DataFrame from the dictionary which is the output of openai api labeling
    
    """
    output_dict = {
        'unique_case_id':[],
        'official_case_id':[],
        'Type of Crime':[],
        'Location':[],
        'Year':[],
        'Date':[],  # Modified
        'Time':[],
        "Victim's Sex":[],
        "Offender's Sex":[],
        'Number of Victims':[],
        'Number of Offenders':[]
}   
    for count, (case_id, case_info) in enumerate(input_dict.items()):
        print(f'======== Extracting info from labeled case number {count+1}: {case_id} ========')
        try:
        #Put infos in a dictionary
#         print(case_info)
            gpt_output_lst = case_info.strip().split('\n')
            gpt_output_lst = [ele.strip('\n') for ele in gpt_output_lst]
            print('gpt_output_lst')
            print(gpt_output_lst)
            output_dict['official_case_id'].append(case_id.zfill(7))
            output_dict['unique_case_id'].append(str(count+1).zfill(6) + '_' + case_id)
            for element in gpt_output_lst:
                key, value = element.split(":")
                if key in output_dict.keys():
                    output_dict[key].append(value.strip())

        
        except:
            print(f'!!======== Errors occurs while procesing case id {case_id} ========!!')
#             # Move file to folder
#             file_name = f'pressemitteilung.{case_id}.json'
#             current_file_path = os.path.join(path_to_json_files, 'info-extracted-cases', file_name)
#             destination_dir_path = os.path.join(path_to_json_files, 'unsuccessful-cases')
            
#             if not os.path.exists(destination_dir_path):
#                 os.makedirs(destination_dir_path)
            
#             shutil.move(current_file_path, os.path.join(destination_dir_path, file_name))
            break
    
    output_dict_renamed = {key.lower().replace(' ', '_').replace("'s", ""): value 
                               for key, value in output_dict.items()}
    
    return output_dict_renamed

In [143]:
def output_csv(path_to_json_files, df):
    output_folder = 'csv-output'
    output_path = os.path.join(path_to_json_files, output_folder)
    cur_datetime = datetime.now().strftime("%Y-%m-%d %H-%M")
    if not os.path.exists(output_path):
            os.makedirs(output_path)
    
    csv_file_name = f'labeled_{df.shape[0]}_cases_{cur_datetime}.csv'
    df.to_csv(os.path.join(output_path, csv_file_name), sep=',', index=False)
    print(f'file save with name: {csv_file_name}')

In [210]:
def main_process(specification_1, specification_2, path_to_json_files, num_case=5, export_csv=False):
    # Input files
    files_gen = sorted(
            glob.glob(
            os.path.join(path_to_json_files, specification_1, specification_2, 'pressemitteilung*')),
            reverse=True)
    
    
    # Main processing
    
    start_time = time.time()  # Measure run time of GPT labeling
    original_dict = {}
    original_dict_filled = label_cases_with_gpt(num_case, files_gen, original_dict)
    end_time = time.time() # Measure run time of GPT labeling
    runtime_minutes = (end_time - start_time) / 60
    print(f"======= GPT labeling runtime: {runtime_minutes:.2f} minutes======")
    info_dict = inject_cases_info_to_dict(original_dict_filled)
    
    df = pd.DataFrame.from_dict(info_dict)
#     print(df)
    
    if export_csv:
        output_csv(path_to_json_files, df)
        
    return df

In [211]:
# def move_file(file_name, path_to_json_files, current_dir, destination_dir):
#         destination_dir_path = os.path.join(path_to_json_files, destination_dir) #'info-extracted-cases')
#         if not os.path.exists(destination_dir_path):
#             os.makedirs(destination_dir_path)
#         shutil.move(file_path, os.path.join(destination_dir_path, file_name))

# Start looping

### One function for all

In [212]:
df_final = main_process('categorized-test-latest-2000-files', 'one-case', path_to_json_files, 40)

======== Working on lebaling case with GPT: 1303317 ========
======== Response.json(): {'error': {'message': "This model's maximum context length is 4097 tokens, however you requested 4718 tokens (2930 in your prompt; 1788 for the completion). Please reduce your prompt; or completion length.", 'type': 'invalid_request_error', 'param': None, 'code': None}} ========
!!!!!======== Errors occurs while labeling case id 1303317 ========!!!!!
======= GPT labeling runtime: 0.01 minutes======


In [213]:
df_final

,unique_case_id,official_case_id,type_of_crime,location,year,date,time,victim_sex,offender_sex,number_of_victims,number_of_offenders


In [206]:
# Save to CSV
output_csv(path_to_json_files, df_final)

file save with name: labeled_34_cases_2023-06-01 19-20.csv


### Funciton seperated

In [207]:
specification_1 = 'categorized-test-latest-2000-files'
specification_2 = 'one-case'

files_gen = sorted(glob.glob(os.path.join(path_to_json_files, specification_1, specification_2, 'pressemitteilung*')), reverse=True)

In [208]:
# files_gen = sorted(
#         glob.glob(
#         os.path.join(path_to_json_files, specification_1, specification_2, 'pressemitteilung*')),
#         reverse=True)

# Main processing
original_dict = {}
original_dict_filled = label_cases_with_gpt(1, files_gen, original_dict)


======== Working on lebaling case with GPT: 1303317 ========
======== Response.json(): {'error': {'message': "This model's maximum context length is 4097 tokens, however you requested 4718 tokens (2930 in your prompt; 1788 for the completion). Please reduce your prompt; or completion length.", 'type': 'invalid_request_error', 'param': None, 'code': None}} ========
!!!!!======== Errors occurs while labeling case id 1303317 ========!!!!!


In [209]:
info_dict = inject_cases_info_to_dict(original_dict_filled)

In [ ]:
df = pd.DataFrame.from_dict(info_dict)
df

In [155]:
original_dict_filled

{'1315085': "\n\nYear: 2023\nDate: 19.04\nTime: Mitternacht\nType of Crime: General Assault, Property Crime, Verbal Abuse\nLocation: Neukölln\nVictim's Sex: Female\nOffender's Sex: Male\nNumber of Victims: 1\nNumber of Offenders: 1"}

In [154]:
info_dict

{'unique_case_id': ['000001_1315085'],
 'official_case_id': ['1315085'],
 'type_of_crime': ['General Assault, Property Crime, Verbal Abuse'],
 'location': ['Neuköll'],
 'year': ['2023'],
 'date': ['19.04'],
 'time': ['Mitternacht'],
 'victim_sex': ['Female'],
 'offender_sex': ['Male'],
 'number_of_victims': ['1'],
 'number_of_offenders': ['1']}

In [589]:
output_folder = 'csv-output'
output_path = os.path.join(path_to_json_files, output_folder)
cur_datetime = datetime.now().strftime("%Y-%m-%d %H-%M")
if not os.path.exists(output_path):
        os.makedirs(output_path)

df.to_csv(os.path.join(output_path, f'labeled_{df.shape[0]}_cases_{cur_datetime}.csv'), sep=',', index=False)

In [ ]:
#TODO:
# One hot encdoing
#Timestamp

In [ ]:
#Next Steps 
# Lena todo: make sure promt works in function for one case - (all cases)
# Hsin todo: loop through cases - output dataframe

## Backup